# Virtual Home Planner using LAPKT

The following provides a complete instruction on generating actions given Virtualhome environment intial setting defined using a pddl file. Please see 'example_problem.pddl'to know how these problem files are defined.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1_FMnK6rmCjVCtQbj18qbuhxJGrIvA7Up?usp=sharing)

# Setup PDDLMultiAgent Planner Directory

In [1]:
!git -c core.symlinks=true clone --recurse-submodules https://github.com/xavierpuigf/PDDLMultiAgent
%cd PDDLMultiAgent

Cloning into 'PDDLMultiAgent'...
remote: Enumerating objects: 197, done.
remote: Total 197 (delta 0), reused 0 (delta 0), pack-reused 197
Receiving objects: 100% (197/197), 210.27 KiB | 4.89 MiB/s, done.
Resolving deltas: 100% (96/96), done.
Submodule 'LAPKT-public' (https://github.com/LAPKT-dev/LAPKT-public.git) registered for path 'LAPKT-public'
Cloning into '/content/PDDLMultiAgent/LAPKT-public'...
remote: Enumerating objects: 11594, done.        
remote: Counting objects: 100% (2147/2147), done.        
remote: Compressing objects: 100% (1263/1263), done.        
remote: Total 11594 (delta 1079), reused 1750 (delta 715), pack-reused 9447        
Receiving objects: 100% (11594/11594), 15.23 MiB | 4.45 MiB/s, done.
Resolving deltas: 100% (6708/6708), done.
Submodule path 'LAPKT-public': checked out 'af757138d0b4d903e35e5946b3b88e8f8d64e976'
/content/PDDLMultiAgent


# Build LAPKT

In [84]:
!sudo apt-get install xutils-dev
!sudo apt install flex 
!sudo apt install bison
!python -m pip install scons

Reading package lists... Done
Building dependency tree       
Reading state information... Done
xutils-dev is already the newest version (1:7.7+5ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
flex is already the newest version (2.6.4-6).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
bison is already the newest version (2:3.0.4.dfsg-1build1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package judy


In [28]:
%cd LAPKT-public/planners/siw_plus-then-bfs_f-ffparser
!scons

/content/PDDLMultiAgent/LAPKT-public/planners/siw_plus-then-bfs_f-ffparser
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
g++ -o src/main.o -c -Wall -std=c++11 -O3 -DNDEBUG -I/content/PDDLMultiAgent/LAPKT-public/src -I/content/PDDLMultiAgent/LAPKT-public/include -I/content/PDDLMultiAgent/LAPKT-public/interfaces/agnostic -I/content/PDDLMultiAgent/LAPKT-public/interfaces/ff-wrapped -I/content/PDDLMultiAgent/LAPKT-public/include -I/content/PDDLMultiAgent/LAPKT-public/external -I/content/PDDLMultiAgent/LAPKT-public/interfaces/agnostic -I/content/PDDLMultiAgent/LAPKT-public/interfaces/ff-wrapped -Isrc src/main.cxx
g++ -o /content/PDDLMultiAgent/LAPKT-public/src/bit_array.o -c -Wall -std=c++11 -O3 -DNDEBUG -I/content/PDDLMultiAgent/LAPKT-public/src -I/content/PDDLMultiAgent/LAPKT-public/include -I/content/PDDLMultiAgent/LAPKT-public/interfaces/agnostic -I/content/PDDLMultiAgent/LAPKT-public/interfaces/ff-wrapped -I/content/PDDLMultiAgent

In [30]:
%cd ../../external/libff
!make clean
!make depend
!make

/content/PDDLMultiAgent/LAPKT-public/external/libff
rm -f *.o *.bak *~ *% core *_pure_p9_c0_400.o.warnings test_lib \
        \#*\#  scan-fct_pddl.tab.c scan-ops_pddl.tab.c scan-probname.tab.c lex.fct_pddl.c lex.ops_pddl.c 
makedepend -- main.c memory.c output.c parse.c expressions.c inst_pre.c inst_easy.c inst_hard.c inst_final.c relax.c search.c utility.c  libff.c memory.c output.c parse.c expressions.c inst_pre.c inst_easy.c inst_hard.c inst_final.c relax.c search.c utility.c scan-fct_pddl.tab.c scan-ops_pddl.tab.c scan-probname.tab.c lex.fct_pddl.c lex.ops_pddl.c  test_main.cxx
makedepend: warning:  main.c (reading /usr/include/stdlib.h, line 25): cannot find include file "bits/libc-header-start.h"
	not in /usr/include/bits/libc-header-start.h
makedepend: warning:  main.c (reading /usr/include/stdlib.h, line 31): cannot find include file "stddef.h"
	not in /usr/include/stddef.h
makedepend: warning:  main.c (reading /usr/include/stdlib.h, line 55): cannot find include file "bits/flo

# Sample Run a planner

In [37]:
root_dir = "/content/PDDLMultiAgent"
%cd $root_dir

## Implement customize siw-then-bfsf planner wrapper

The planner wrapper from the repo does not work as it depends on a symlink on github. 

In [59]:
import subprocess
import os
import pdb
import utils_env_parser
import urllib.request as urllibreq
import urllib
import argparse
import json, sys
import time

In [80]:
planner_pth = os.path.join(root_dir, 'LAPKT-public', 'planners', 'siw_plus-then-bfs_f-ffparser' ,'siw-then-bfsf')
def planner(domain_name, problem_name, file_out, planner_dir = planner_pth):
    start = time.time()
    stats_dir = '/'.join(file_out.split('/')[:-3]) + '/stats/' 
    file_stats = stats_dir + file_out.split('/')[-1] + '_stats.txt'
    os.makedirs(stats_dir, exist_ok=True)
    with open(file_stats, 'w+') as outfile:
        proc = subprocess.call([planner_pth, '--domain', domain_name, '--problem', problem_name, '--output', file_out], stdout=outfile)
    with open(file_out, 'r') as f:
        program = f.readlines()
        if len(program) > 0 and len(program[-1]) == 0:
            program = program[:-1]
    program = utils_env_parser.convert_to_virtualhome_program(program)
    program = [x+'\n' for x in program]
    if len(program) > 0:
        with open(file_out, 'w+') as f:
            f.writelines(program)
        print("Successfully executed the planner.")
    else:
        os.remove(file_stats)
        os.remove(file_out)
        print("Planner has no output.")
    end = time.time()
    print("Time used: {}".format(end-start))


In [89]:
! ./LAPKT-public/planners/siw_plus-then-bfs_f-ffparser/siw-then-bfsf --domain domain.pddl --problem example_problem/example_problem_short.pddl --output file_out



task contains conditional effects. turning off state domination.

 --- OK.
 Match tree built with 11 nodes.

PDDL problem description loaded: 
	Domain: VIRTUALHOME
	Problem: GOTOKITCHEN
	#Actions: 11
	#Fluents: 30
Landmarks found: 1
Starting search with IW (time budget is 60 secs)...
rel_plan size: 1
#RP_fluents 4
Caption
{#goals, #UNnachieved,  #Achieved} -> IW(max_w)

{1/1/0}:IW(1) -> [2]rel_plan size: 0
#RP_fluents 0Plan found with cost: 1
Total time: 0.000111
Nodes generated during search: 2
Nodes expanded during search: 1
IW search completed


In [90]:
domain_name='./domain.pddl'
problem_name='./example_problem/example_problem_short.pddl'
output_dir = "./example_out"
planner(domain_name=domain_name, problem_name=problem_name, file_out=output_dir)

Successfully executed the planner.
Time used: 0.020502090454101562


In [91]:
#printout result
f = open('./example_out')
for line in f:
    print(line)

[WALK] <KITCHEN> (11)

